In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import requests
import tensorflow as tf
import string

In [3]:
response = requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt')

In [4]:
data = response.text.split('\n')
data = data[253:]
data[0]

'  From fairest creatures we desire increase,'

In [5]:
len(data)

124204

In [6]:
data = " ".join(data)

In [7]:
def clean_text(doc):
    tokens = doc.split() #create tokens based on whitepace if more than one whitespace then a whitespace is treated as a token
    translation_table = str.maketrans('', '', string.punctuation) #get all punctuation
    tokens = [w.translate(translation_table) for w in tokens] #remove punctiation from strings and append them to the tokens list
    tokens = [word for word in tokens if word.isalpha()] #append only words which are alphabets
    tokens = [word.lower() for word in tokens] #make all word lowercase
    return tokens

In [8]:
tokens = clean_text(data)

In [9]:
tokens[:10]

['from',
 'fairest',
 'creatures',
 'we',
 'desire',
 'increase',
 'that',
 'thereby',
 'beautys',
 'rose']

In [10]:
len(tokens)

898199

In [11]:
len(set(tokens)) #total number of unique words

27956

In [12]:
length = 50 + 1 #total number of input word 1 is for prediction(output)

In [13]:
lines =[]

for i in range(length, len(tokens)):
    seq = tokens[i-length:i] #take tokens from index 0 to 51
    line = ' '.join(seq) #join all the tokens in the seq to make a line 
    lines.append(line)
    if i > 50000:
        break  #help in training with low computing power to only read 200k tokens

print(len(lines))

49951


In [14]:
lines[0] #first ine with 51 tokens

'from fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self'

In [15]:
lines[1] #does not have "from"

'fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self thy'

In [16]:
# spplit X and y
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [17]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [18]:
sequences = np.array(sequences)

In [19]:
X, y = sequences[:, :-1], sequences[:,-1] #sequences[:, :-1] : all rows :-1 all columns except the last sequences[:,-1] all rows and the last column

In [20]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

6029

In [21]:
y = to_categorical(y, num_classes = vocab_size)

In [22]:
y[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [23]:
seq_len = X.shape[1]

In [24]:
seq_len

50

In [25]:
#Building Model

model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_len))
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

2022-05-03 14:14:05.658523: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-03 14:14:05.762249: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-03 14:14:05.763394: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-03 14:14:05.765178: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            301450    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 200)           200800    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               120400    
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 6029)              608929    
Total params: 1,241,679
Trainable params: 1,241,679
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
model.fit(X, y, batch_size=64, epochs =100, verbose = 1)

2022-05-03 14:14:09.269433: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1204618316 exceeds 10% of free system memory.
2022-05-03 14:14:10.631188: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1204618316 exceeds 10% of free system memory.
2022-05-03 14:14:11.702465: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2022-05-03 14:14:16.343578: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


781/781 [==============================] - 16s 13ms/step - loss: 6.8332 - accuracy: 0.0250
Epoch 2/100
781/781 [==============================] - 10s 13ms/step - loss: 6.4876 - accuracy: 0.0345
Epoch 3/100
781/781 [==============================] - 10s 13ms/step - loss: 6.3327 - accuracy: 0.0399
Epoch 4/100
781/781 [==============================] - 10s 13ms/step - loss: 6.1943 - accuracy: 0.0490
Epoch 5/100
781/781 [==============================] - 10s 13ms/step - loss: 6.0571 - accuracy: 0.0576
Epoch 6/100
781/781 [==============================] - 10s 13ms/step - loss: 5.9410 - accuracy: 0.0616
Epoch 7/100
781/781 [==============================] - 11s 14ms/step - loss: 5.8408 - accuracy: 0.0661
Epoch 8/100
781/781 [==============================] - 10s 13ms/step - loss: 5.7441 - accuracy: 0.0712
Epoch 9/100
781/781 [==============================] - 10s 13ms/step - loss: 5.6623 - accuracy: 0.0776
Epoch 10/100
781/781 [==============================] - 10s 13ms/step - loss: 5.5756 

In [29]:
def generate_text(model, tokenizer, text_seq_length, seed_text, n_words):
    text = []
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen=text_seq_length, truncating='pre')
        
        y_predict = np.argmax(model.predict(encoded))
        
        predicted_word = ''
        for word, index in tokenizer.word_index.items():
            if index == y_predict:
                predicted_word = word
                break
        seed_text = seed_text + ' ' + predicted_word
        text.append(predicted_word)
    return ' '.join(text)    
    
        

In [30]:
seed_text = lines[15344]
seed_text

'to thy will one will of mine to make thy large will more let no unkind no fair beseechers kill think all but one and me in that one will if thy soul check thee that i come so near swear to thy blind soul that i was thy will and'

In [31]:
generate_text(model, tokenizer, seq_len, seed_text, 100)

'will thou use my heart with such a suit are turned maiden flattery steals the best of more her and as too great so error and lack the property by every fear upon your vows to caesar is all my mind thy motive is not well rash and said the honour by your pleasure loves it is famished for a most state lose both sides what is black do practise upon him and me be much the prince of mine ear with their cuckold they in herself look infects the benefit of rest and bristly beard in all and time'